In [22]:
from utils import *

In [3]:
from nilearn.signal import clean

In [4]:
#open pickle file
path = "data/data_path_dictionary.pkl"
data_path_dictionary = open_pickle(path)

print("Dictionary contents: ")
data_path_dictionary

Dictionary contents: 


defaultdict(list,
            {'subject_data': ['svm_data/10004_08693/svm_subj_vecs.mat',
              'svm_data/10008_09924/svm_subj_vecs.mat',
              'svm_data/10009_08848/svm_subj_vecs.mat',
              'svm_data/10016_09694/svm_subj_vecs.mat',
              'svm_data/10017_08894/svm_subj_vecs.mat',
              'svm_data/10018_08907/svm_subj_vecs.mat',
              'svm_data/10021_08839/svm_subj_vecs.mat',
              'svm_data/10022_08854/svm_subj_vecs.mat',
              'svm_data/10023_09126/svm_subj_vecs.mat',
              'svm_data/10027_09455/svm_subj_vecs.mat',
              'svm_data/10033_08871/svm_subj_vecs.mat',
              'svm_data/10034_08879/svm_subj_vecs.mat',
              'svm_data/10035_08847/svm_subj_vecs.mat',
              'svm_data/10036_09800/svm_subj_vecs.mat',
              'svm_data/10037_09903/svm_subj_vecs.mat',
              'svm_data/10038_09063/svm_subj_vecs.mat',
              'svm_data/10039_08941/svm_subj_vecs.mat',
              

In [5]:
# single subject data paths from S3
masks_path = data_path_dictionary['mask_data'][0]
label_path = data_path_dictionary['labels'][0]

In [6]:
%%time
# Get mat data from aws
mask_data = access_load_data(masks_path)
mask_data.keys()

CPU times: total: 375 ms
Wall time: 1.82 s


dict_keys(['__header__', '__version__', '__globals__', 'mask', 'masksubACC', 'masksubAI', 'masksubNAcc', 'masksubmPFC'])

In [7]:
%%time
# Get label data from aws
label_data = access_load_data(label_path)
label_data.keys()

CPU times: total: 78.1 ms
Wall time: 1.31 s


dict_keys(['__header__', '__version__', '__globals__', 'decrease_onsets', 'increase_onsets', 'rest_ons', 'rt_labels'])

In [9]:
# Takes in filepaths and mask type and returns a numpy-compatible mask

mask_data_filepath = data_path_dictionary['mask_data'][0]
mask_type = 'mask'

mask = create_mask(mask_data_filepath, mask_type)
mask

array([False, False, False, ..., False, False, False])

In [20]:
def labels_mask_binary_update(label_data_path, label_type = 'rt_labels'):
    """

    """
    # Get Mask data
    label_data_dict = access_load_data(label_data_path)
    labels = label_data_dict[label_type]
    mask_labels_indices = np.where(labels != 9999)
    mask_labels_indices = mask_labels_indices[0]

    # Get Binary Labels
    label_mask = np.all(labels != 9999, axis=1)
    binary_labels = labels.reshape(-1)[label_mask]
    binary_labels

    return mask_labels_indices, binary_labels



masked_indices,binary_labels = labels_mask_binary_update(label_path)

In [28]:
subject_path = data_path_dictionary['subject_data'][0] #'svm_data/10004_08693/svm_subj_vecs.mat'
subject = access_load_data(subject_path)
subject

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Wed Feb 23 06:31:07 2022',
 '__version__': '1.0',
 '__globals__': [],
 'run_04_vec': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int16),
 'run_03_vec': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int16),
 'run_02_vec': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int16),
 'run_01_vec': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 

In [33]:
def masking_data_me(subject, mask, mask_labels, binary_labels,do_norm):
    """


    """

    arr = []
    label_arr = []
    for i in tqdm.tqdm(range(4)):
        user_key = 'run_0' + str(i+1) + '_vec'
        array = subject[user_key]
        array_masked = array[:, mask]
        array_masked = array_masked[mask_labels]

        # Add function for standard scaler z-score normalization
        # across each run
        #scaler = StandardScaler.fit(array_masked[mask_labels])
        #masked_norm_data = scaler.transform(array_masked[mask_labels])


        #arr.append(masked_norm_data)
        if do_norm:
            array_masked = clean(array_masked,standardize='psc')

        arr.append(array_masked)
        label_arr.append(binary_labels)

    return arr, label_arr

In [35]:
subj_mask_data = masking_data_me(subject,mask,masked_indices,binary_labels,True)

100%|██████████| 4/4 [00:01<00:00,  2.04it/s]


In [36]:
subj_mask_data

([array([[ 0.       ,  0.       ,  0.       , ...,  3.4595332, -2.7081203,
          -4.08065  ],
         [ 0.       ,  0.       ,  0.       , ...,  3.0718458, -2.8591168,
          -2.5701163],
         [ 0.       ,  0.       ,  0.       , ...,  1.8663439, -4.98893  ,
          -3.2585096],
         ...,
         [ 0.       ,  0.       ,  0.       , ...,  3.3766582,  4.5196137,
           6.078048 ],
         [ 0.       ,  0.       ,  0.       , ..., -2.0088224, -1.2887629,
          -3.3319068],
         [ 0.       ,  0.       ,  0.       , ...,  1.0262262, -3.976704 ,
          -2.5131683]], dtype=float32),
  array([[ 0.        ,  0.        ,  0.        , ...,  3.8679485 ,
           0.96662223, -2.143643  ],
         [ 0.        ,  0.        ,  0.        , ...,  1.5796369 ,
           6.191506  ,  6.2866488 ],
         [ 0.        ,  0.        ,  0.        , ...,  5.3922853 ,
          -1.475869  , -2.6557198 ],
         ...,
         [ 0.        ,  0.        ,  0.        , ..., -